#### Import dependancies and set environment determinism

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
#     tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(
    0
)
import numpy as np
import random
import pandas as pd


SEED = 378
# SEED = 123
print(SEED)
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

In [ ]:
import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tqdm import tqdm

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [ ]:
import path_manager as pm
[
        home_path, data_path, RGI_path, glathida_path, 
        ref_path, coregistration_testing_path, 
        arch_test_path, LOO_path
] = pm.set_paths()

#### define SNN

In [ ]:
def run_model(model_path,l1,l2,loss):
            
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat))

    model = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1, layer_2 = l2,loss = loss
    )

    model_history = model.fit(
        trfeat,
        trlabs,
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
    model_filename = os.path.join(model_path)
    model.save(model_filename)
    return model

#### Create callback function to quit training if loss does not improve after 10 iterations

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None,
    restore_best_weights = True
)

#### Define path to save arch test models

In [ ]:
l1_list = [3,4,5,6,7]
l2_list = [2,3,4,5]
loss_list = ['mse','mae']
df = gl.coregister_data(data_path,'4')


dft = df.copy()

trfeat, tefeat, trlabs, telabs = gl.split_data(df)
for loss in loss_list:
    for l2 in l2_list:
        for l1 in l1_list:
            if l2 >= l1:
                continue
            else:

                model = {}
                model_history = {}
                normalizer = {}
#                 path = '/data/fast1/glacierml/models/LOO_loss_testing/'
                model_path = os.path.join(
                   arch_test_path ,loss,str(l1)+'-'+str(l2)
                )
                res_dir = os.path.join(arch_test_path, loss,'final_results.pkl')
                if os.path.isdir(res_dir) == True:
                    continue

                isdir = model_path
                if os.path.isdir(isdir) == True:
                    continue
                if os.path.isdir(isdir) == False:
                    print(model_path)
                    set_global_determinism(seed=SEED)
                    run_model(model_path,l1,l2,loss)
print('all done')